## Data Preparation and Feature Engineering
This notebook prepares all the required datasets for modeling NBA playoff game outcomes.

#### Build Regular Season Team Statistics 
In this section, we prepare team statistics for the NBA regular season from 2015 to 2025. These features capture team performance using advanced metrics and are essential for building playoff game prediction models.

##### Overview of Steps:
1. **Load Data**: Read `Games.csv` and `TeamStatistics.csv` files.
2. **Assign Season**: Derive the NBA season (e.g., 2021–2022) based on game date.
3. **Filter Regular Season Games**: Keep only games labeled as "Regular Season".
4. **Calculate Four Factors**:
   - Effective Field Goal % (`eFG%`)
   - Turnover Rate (`TOV%`)
   - Free Throws Per Field Goal Attempt (`FT/FGA`)
   - Offensive Rebound % (`ORB%`)
   - Defensive Rebound % (`DRB%`)
5. **Merge Opponent Rebounds**: Bring in opposing team rebound data to compute `ORB%` and `DRB%`.
6. **Format Team Names**: Combine city and team name into a single string (e.g., `"Atlanta Hawks"`).
7. **Aggregate Stats**: Average all metrics by team and season.
8. **Filter by Years**: Keep only seasons from 2015 to 2025.


In [3]:
# Import required libraries
import pandas as pd

# Load games and team statistics data
games_df = pd.read_csv("../data/raw/Games.csv", low_memory=False)
team_stats_df = pd.read_csv("../data/raw/TeamStatistics.csv", low_memory=False)

# Assign a season to each game
def get_season_year(date_str):
    date = pd.to_datetime(date_str)
    return date.year + 1 if date.month >= 10 else date.year

team_stats_df["season"] = team_stats_df["gameDate"].apply(get_season_year)

# Filter for regular season games
# Merge gameType info from games_df
merged = pd.merge(
    team_stats_df,
    games_df[["gameId", "gameType"]],
    on="gameId",
    how="left"
)

# Keep only regular season games
df = merged[merged["gameType"].str.strip() == "Regular Season"].copy()

# Compute the Four Factors team statistics
df["eFG%"] = (df["fieldGoalsMade"] + 0.5 * df["threePointersMade"]) / df["fieldGoalsAttempted"]
df["TOV%"] = df["turnovers"] / (df["fieldGoalsAttempted"] + 0.44 * df["freeThrowsAttempted"] + df["turnovers"])
df["FT/FGA"] = df["freeThrowsMade"] / df["fieldGoalsAttempted"]

# Merge opponent rebounds for ORB% and DRB%
opp_stats = df[["gameId", "teamId", "reboundsOffensive", "reboundsDefensive"]].copy()
opp_stats.columns = ["gameId", "opponentTeamId", "oppORB", "oppDRB"]

df = pd.merge(df, opp_stats, on=["gameId", "opponentTeamId"], how="left")

df["ORB%"] = df["reboundsOffensive"] / (df["reboundsOffensive"] + df["oppDRB"]).clip(lower=1)
df["DRB%"] = df["reboundsDefensive"] / (df["reboundsDefensive"] + df["oppORB"]).clip(lower=1)

# Combine team city and team name
df["teamName"] = df["teamCity"].str.strip() + " " + df["teamName"].str.strip()

# Group by team name and season to compute season averages
regular_season_stats = df.groupby(["teamName", "season"])[["eFG%", "TOV%", "FT/FGA", "ORB%", "DRB%"]].mean().reset_index()

# Filter to include only NBA seasons from 2015 to 2025
regular_season_stats = regular_season_stats[(regular_season_stats["season"] >= 2015) & (regular_season_stats["season"] <= 2025)].dropna()

# Export the final dataset
regular_season_stats.to_csv("../data/processed/team_statistics_regular_season.csv", index=False)

#### Build Playoff Game Features
In this section, we extract and prepare the data for all NBA playoff games from 2015 to 2025. This includes identifying the season, playoff round, matchup type, game number, seeds, and whether the home team had home-court advantage.

##### Overview of Steps:
1. **Load Data**: Read `Games.csv` file.
2. **Filter for Playoffs**: Keep only rows where `gameType` is `"Playoffs"`.
3. **Assign Season**: Derive the NBA season (e.g., 2021–2022) based on `gameDate`.
4. **Format Team Names**: Combine city and name for `homeTeam` and `awayTeam`.
5. **Create Series ID**: Combine `homeTeam`, `awayTeam`, and `season` into a unique ID per playoff series.
6. **Normalize Round**: Convert `gameLabel` (e.g., "NBA Finals") into a numerical round indicator (1–4).
7. **Extract Game Info**: Assign `seriesGameNumber` (default to 1) and flag if the `homeTeam` won.
8. **Merge Seeds**: Join seed info from `playoff_seeds_2015_2025.csv` for both home and away teams.
9. **Add Matchup & Home-Court Flags**:
   - Generate a readable `matchupType` (e.g., `"Conference Semifinals"`)
   - Set `homeCourt = True` if `homeSeed < awaySeed`

In [4]:
# Import required libraries
import pandas as pd

# Load games data
games_df = pd.read_csv("../data/raw/Games.csv", low_memory=False)

# Filter for playoff games only
playoff_games = games_df[games_df["gameType"].str.strip() == "Playoffs"].copy()

# Assign a season to each playoff game
def get_season_year(date_str):
    date = pd.to_datetime(date_str)
    return date.year + 1 if date.month >= 10 else date.year

playoff_games["season"] = playoff_games["gameDate"].apply(get_season_year)

# Format home and away team names
playoff_games["homeTeam"] = playoff_games["hometeamCity"].str.strip() + " " + playoff_games["hometeamName"].str.strip()
playoff_games["awayTeam"] = playoff_games["awayteamCity"].str.strip() + " " + playoff_games["awayteamName"].str.strip()

# Create a unique series ID based on matchup and season
playoff_games["seriesId"] = (
    playoff_games[["homeTeam", "awayTeam", "season"]]
    .astype(str)
    .agg("_vs_".join, axis=1)
)

# Normalize game round into a number (1 = First Round, ..., 4 = NBA Finals)
def get_round(label):
    if not isinstance(label, str):
        return None
    label = label.lower()
    if "first round" in label:
        return 1
    elif "semifinal" in label:
        return 2
    elif "conf. finals" in label or "conference finals" in label:
        return 3
    elif "nba finals" in label and "conf" not in label:
        return 4
    return None

playoff_games["round"] = playoff_games["gameLabel"].apply(get_round)

# Extract game number in series and whether the home team won
playoff_games["gameNumber"] = playoff_games["seriesGameNumber"].fillna(1).astype(int)
playoff_games["homeWin"] = playoff_games["winner"] == playoff_games["hometeamId"]

# Select relevant metadata columns
playoff_metadata = playoff_games[[
    "gameId", "gameDate", "season", "homeTeam", "awayTeam",
    "seriesId", "round", "gameNumber", "homeWin", "gameLabel",
    "gameSubLabel", "homeScore", "awayScore"
]]

# Merge home and away team seeds
games = (
    playoff_metadata
    .merge(
        pd.read_csv("../data/raw/playoff_seeds_2015_2025.csv")
          .rename(columns={"team": "homeTeam", "seed": "homeSeed"}),
        on=["season", "homeTeam"],
        how="left"
    )
    .merge(
        pd.read_csv("../data/raw/playoff_seeds_2015_2025.csv")
          .rename(columns={"team": "awayTeam", "seed": "awaySeed"}),
        on=["season", "awayTeam"],
        how="left"
    )
)

# Create readable matchup name
round_name_map = {
    1: "First Round",
    2: "Conference Semifinals",
    3: "Conference Finals",
    4: "NBA Finals"
}
games["matchupType"] = games["round"].map(round_name_map)

# Rename conference columns (if present)
games["conference_home"] = games.get("conference_x")
games["conference_away"] = games.get("conference_y")
games = games.drop(columns=["conference_x", "conference_y"], errors="ignore")

# Calculate home-court advantage (homeSeed < awaySeed)
games["homeCourt"] = games["homeSeed"] < games["awaySeed"]

# Export final dataset
games.to_csv("../data/processed/playoffs_games.csv", index=False)

#### Build Head-to-Head Regular Season Statistics
In this section, we calculate regular season head-to-head statistics between each pair of playoff opponents prior to their matchup. These features help capture matchup history and momentum heading into the playoffs.

##### Overview of Steps:
1. **Load Data**: Import both `Games.csv` and `TeamStatistics.csv`.
2. **Assign Season**: Determine NBA season for each game based on the `gameDate`.
3. **Filter for Regular Season**: Keep only games where `gameType == "Regular Season"` and season is between 2015 and 2025.
4. **Format Team Names**: Clean and standardize both `teamName` and `opponentName` using `teamCity` and `teamName` fields.
5. **Create Win Flag**: Add a binary column to indicate whether the team won each matchup.
6. **Aggregate Head-to-Head Stats**:
   - Count how many times each team faced a specific opponent during the regular season.
   - Count how many of those games resulted in a win for that team.
   - Calculate the regular season win rate against that opponent.

In [7]:
# Import libraries
import pandas as pd

# Load games dataset
games_df = pd.read_csv("../data/raw/Games.csv", low_memory = False)
stats_df = pd.read_csv("../data/raw/TeamStatistics.csv", low_memory = False)

# Assign a season to each game
def get_season_year(date_str):
    date = pd.to_datetime(date_str)
    return date.year + 1 if date.month >= 10 else date.year

stats_df["season"] = stats_df["gameDate"].apply(get_season_year)

# Merge to get gameType info
games_df_small = games_df[["gameId", "gameType"]]
merged_df = stats_df.merge(games_df_small, on="gameId", how="left")

# Filter for Regular Season Games Only (2015-2025)
regular_season = merged_df[merged_df["gameType"] == "Regular Season"].copy()
regular_season = regular_season[(regular_season["season"] >= 2015) & (regular_season["season"] <= 2025)]

# Standardize Team and Opponent Names
regular_season["teamName"] = regular_season["teamCity"].str.strip() + " " + regular_season["teamName"].str.strip()
regular_season["opponentName"] = regular_season["opponentTeamCity"].str.strip() + " " + regular_season["opponentTeamName"].str.strip()

# Create Win Flag
regular_season["win_flag"] = regular_season["win"] == 1

# Group and aggregate head-to-head statistics
head_to_head = regular_season.groupby(["season", "teamName", "opponentName"]).agg(
    games_played=("gameId", "count"),
    wins=("win_flag", "sum")
).reset_index()
head_to_head["losses"] = head_to_head["games_played"] - head_to_head["wins"]
head_to_head["win_rate"] = head_to_head["wins"] / head_to_head["games_played"]

# Export final dataset
head_to_head.to_csv("../data/processed/regular_season_head_to_head_stats.csv", index=False)

#### Build Playoff Game Team Statistics
This section processes and calculates team-level playoff statistics **for each game** between 2015–2025. These stats will serve as the base inputs for modeling playoff predictions in later sections.

##### Overview of Steps:
1. **Load Data**: Read `Games.csv` and `TeamStatistics.csv` files.
2. **Filter for Playoffs**: Keep only rows where `gameType` is `"Playoffs"`.
3. **Assign Season**: Derive the NBA season (e.g., 2021–2022) based on `gameDate`.
4. **Calculate Four Factors**:
   - Effective Field Goal % (`eFG%`)
   - Turnover Rate (`TOV%`)
   - Free Throws Per Field Goal Attempt (`FT/FGA`)
   - Offensive Rebound % (`ORB%`)
   - Defensive Rebound % (`DRB%`)
5. **Merge Opponent Rebounds**: Needed for computing ORB% and DRB%.
6. **Format Team Names**: Combine city and name for consistency.
7. **Attach Scores**: Merge in home/away scores for each team.
6. **Format Team Names**: Combine city and team name into a single string (e.g., `"Atlanta Hawks"`).
7. **Aggregate Stats**: Average all metrics by team and season.
8. **Filter by Years**: Keep only seasons from 2015 to 2025.

In [11]:
# Import libraries
import pandas as pd

# Load Team Statistics and Games dataset
df = pd.read_csv("../data/raw/TeamStatistics.csv", low_memory=False)

# Add gameType by merging with games.csv (if needed)
games_df = pd.read_csv("../data/raw/Games.csv", low_memory=False)
df = pd.merge(df, games_df[["gameId", "gameType"]], on="gameId", how="left")

# Filter only playoff games
df = df[df["gameType"].str.strip() == "Playoffs"].copy()

# Assign each playoffs game to a season
def get_season_year(date_str):
    date = pd.to_datetime(date_str)
    return date.year + 1 if date.month >= 10 else date.year

df["season"] = df["gameDate"].apply(get_season_year)

# Compute the Four Factors team statistics
df["eFG%"] = (df["fieldGoalsMade"] + 0.5 * df["threePointersMade"]) / df["fieldGoalsAttempted"]
df["TOV%"] = df["turnovers"] / (df["fieldGoalsAttempted"] + 0.44 * df["freeThrowsAttempted"] + df["turnovers"])
df["FT/FGA"] = df["freeThrowsMade"] / df["fieldGoalsAttempted"]

# Merge opponent rebounds for ORB% and DRB%
opp_stats = df[["gameId", "teamId", "reboundsOffensive", "reboundsDefensive"]].copy()
opp_stats.columns = ["gameId", "opponentTeamId", "oppORB", "oppDRB"]

df = pd.merge(df, opp_stats, on=["gameId", "opponentTeamId"], how="left")

df["ORB%"] = df["reboundsOffensive"] / (df["reboundsOffensive"] + df["oppDRB"])
df["DRB%"] = df["reboundsDefensive"] / (df["reboundsDefensive"] + df["oppORB"])

# Combine team city and team name
df["teamName"] = df["teamCity"].str.strip() + " " + df["teamName"].str.strip()

# Identify home and away teams with their scores
score_df = df[["gameId", "teamName", "teamScore", "home"]].copy()

# Separate into home and away team scores
home_scores = score_df[score_df["home"] == True][["gameId", "teamName", "teamScore"]].rename(columns={"teamScore": "homeScore", "teamName": "homeTeam"})
away_scores = score_df[score_df["home"] == False][["gameId", "teamName", "teamScore"]].rename(columns={"teamScore": "awayScore", "teamName": "awayTeam"})

# Merge them back together
game_scores = pd.merge(home_scores, away_scores, on="gameId", how="inner")

# Merge 'game_scores' into playoff team statistics Dataframe
df = df.merge(game_scores[["gameId", "homeScore", "awayScore"]], on="gameId", how="left")

# Filter to include only NBA seasons from 2015 to 2025
df = df[(df["season"] >= 2015) & (df["season"] <= 2025)].copy()

# Keep final playoff statistics per team per 
playoff_ff = df[[
    "gameId", "season", "teamName", "eFG%", "TOV%", "FT/FGA", "ORB%", "DRB%","homeScore","awayScore"
]].copy()

# Export the final dataset
playoff_ff.to_csv("../data/processed/team_statistics_playoff_games.csv", index=False)